# Modelling sun and sky light sources

- This notebook explain how to use astk functions to simulate the natural lighting of a scene.
- This consist of defining a finite set of direction (i.ie. a set of elevation, azimuth) + a set irradiances representing the sky and the sun for a given period.
- It is generally easier to compute the relative irradiance of sources (so that their sum equal 1), and to multiply by mean irradiance or total amount of light depending on the application. This approach is the one demonstrated here.

## Conventions

 - Radiation fluxes are expressed in terms of irradiance of a horizontal unit placed at earth surface(total energetic flux passing through a square meter of ground). 
 - The standard measurement of radiation at earth surface is the global horizontal irradiance (ghi, W.m-2 i.e. J.m-2.s-1), that captures radiations coming from the sun and from the sky in the shortwave domain (visible light)
 - The sun contribution is called direct normal irradiance (dni), whereas sky contribution is called diffuse horizontal irradiance (dhi)

## Imports and setup location

In [1]:
%gui qt
import alinea.astk
from alinea.astk.data_access import montpellier_spring_2013
from alinea.astk.meteorology.sky_irradiance import sky_irradiances
from alinea.astk.sun_and_sky import sky_sources, sun_sky_sources

Specify a location on earth:

In [2]:
Montpellier ={
'longitude': 3.87,
'latitude': 43.61,
'altitude': 56,
'timezone': 'Europe/Paris'}

In [3]:
Montpellier

{'longitude': 3.87,
 'latitude': 43.61,
 'altitude': 56,
 'timezone': 'Europe/Paris'}

## Simulating difuse light conditions

   - When the sky is very cloudy, direct light equals zero, and diffuse light is almost uniform among sky sources as long as cloudy condition last. The absolute value of global horizontal irradiance is varying with time or between two days, but the relative contributions of the different sky sources to ghi does not vary.
   - The positioning and the relative contribution of sky sources to global horizontal irradiance for cloudy conditions can be obtained by the following function that used the 'standard overcast condition' sky luminance model, and make the integration of luminance so that one source capture the portion of sky arround it:

In [4]:
sky = sky_sources(turtle_sectors=16)
#skys.show_sources(sky)
elevation, azimuth, irradiance = sky
irradiance

array([0.15986923, 0.04397197, 0.04397197, 0.04397197, 0.04397197,
       0.04397197, 0.11305245, 0.11305245, 0.11305245, 0.11305245,
       0.11305245, 0.01100174, 0.01100174, 0.01100174, 0.01100174,
       0.01100174])

  - turtle sector indicate the number of sources sampling the sky hemisphere (possible values are 6, 16 and 46)

In [5]:
sum(irradiance)

1.0000000000000002

- Lighting the scene with such light is done like before, yielding the distribution of irradiance within the plant

In [6]:
#cscene, raw, agg = ltfs.illuminate(lscene, light=sky, scene_unit='cm')
#scene, _ = cscene.plot(raw, maxval=1.01)
#skys.show_sources(sky,scene=scene, distance=15, radius=0.3)

* Muliplying the results by ghi (instantaneous or averaged over time) yields absolute irradiance values on organs

In [8]:
 sky_irr = sky_irradiances(daydate='2000-06-21', attenuation=0.25, **Montpellier)
sky_irr.ghi.mean(),sky_irr.dhi.mean()

(137.94282490295114, 137.18468003683014)

In [9]:
sky_irr

,azimuth,zenith,elevation,clearness,brightness,ghi,dni,dhi
2000-06-21 07:00:00+02:00,65.222522,81.781574,8.218426,1.000000,0.087529,17.402057,0.000000,17.402057
2000-06-21 08:00:00+02:00,74.723897,71.647009,18.352991,1.000000,0.143055,60.435420,0.000000,60.435420
2000-06-21 09:00:00+02:00,84.230207,61.003100,28.996900,1.000000,0.165523,107.069357,0.000000,107.069357
2000-06-21 10:00:00+02:00,94.525066,50.169169,39.830831,1.000000,0.176735,150.817077,0.000000,150.817077
2000-06-21 11:00:00+02:00,106.936113,39.517644,50.482356,1.002645,0.182443,187.903059,0.664856,187.390170
2000-06-21 12:00:00+02:00,124.101947,29.712676,60.287324,1.004560,0.185581,215.517829,1.120370,214.544764
2000-06-21 13:00:00+02:00,150.974668,22.305842,67.694158,1.013752,0.185633,231.659389,3.336463,228.572590
2000-06-21 14:00:00+02:00,189.036867,20.362593,69.637407,1.014207,0.185897,235.177442,3.449277,231.943712
2000-06-21 15:00:00+02:00,222.847350,25.217800,64.782200,1.005178,0.186591,225.822742,1.266676,224.676786
2000-06-21 16:00:00+02:00,244.565120,34.007293,55.992707,1.003821,0.184385,204.259004,0.946751,203.474179


## Simulating clear sky conditions

* On clear days, 80% of the radiation is coming from the sun, that is from the south-oriented hemisphere. Moreover, sun irradiance is also varying as a function of time of the day.
* In such case, it is recomended to add sun sources in addition to sky sources. Generaly one position per hour is added 

In [ ]:
sky_irr, sun, sky = sun_sky_sources(daydate='2000-06-21', **Montpellier) 
#sources = skys.add_sources(sun, sky)

#
#cscene, raw, agg = ltfs.illuminate(lscene, light=sources, scene_unit='cm', north=180)
#scene, _ = cscene.plot(raw)
#skys.show_sources(sources,scene=scene, distance=15, radius=0.3, north=180)

- north is the angle (deg, positive clockwise) from X+ to North. It should be pass both to illuminate AND show_source function

## Simulating actual irradiances

In general, meteorological conditions are between overcast and clear sky conditions. The cursor between theses two extremes is a function of the ratio between actual irradiance (measured) and clear sky irradiance. 

get some meteorological data

In [3]:
meteo_data = montpellier_spring_2013()

In [4]:
meteo_data

,ghi
Unnamed: 0,
2013-05-21 00:00:00+02:00,0.000000
2013-05-21 01:00:00+02:00,0.000000
2013-05-21 02:00:00+02:00,0.000000
2013-05-21 03:00:00+02:00,0.000000
2013-05-21 04:00:00+02:00,0.000000
...,...
2013-07-04 19:00:00+02:00,270.550000
2013-07-04 20:00:00+02:00,103.850000
2013-07-04 21:00:00+02:00,6.016667


In [11]:
observed = skys.actual_irradiance('2013-05-26', meteo_data)
observed

2013-05-26 00:00:00+02:00      0.000000
2013-05-26 01:00:00+02:00      0.000000
2013-05-26 02:00:00+02:00      0.000000
2013-05-26 03:00:00+02:00      0.000000
2013-05-26 04:00:00+02:00      0.000000
2013-05-26 05:00:00+02:00      0.000000
2013-05-26 06:00:00+02:00     31.615385
2013-05-26 07:00:00+02:00    166.370370
2013-05-26 08:00:00+02:00    335.425926
2013-05-26 09:00:00+02:00    505.481481
2013-05-26 10:00:00+02:00    660.689655
2013-05-26 11:00:00+02:00    783.516667
2013-05-26 12:00:00+02:00    865.300000
2013-05-26 13:00:00+02:00    887.883333
2013-05-26 14:00:00+02:00    843.233333
2013-05-26 15:00:00+02:00    861.366667
2013-05-26 16:00:00+02:00    700.766667
2013-05-26 17:00:00+02:00    576.966667
2013-05-26 18:00:00+02:00    415.766667
2013-05-26 19:00:00+02:00    241.550000
2013-05-26 20:00:00+02:00     68.566667
2013-05-26 21:00:00+02:00      1.433333
2013-05-26 22:00:00+02:00      0.000000
2013-05-26 23:00:00+02:00      0.000000
2013-05-27 00:00:00+02:00      0.000000


In [12]:
sun, sky = skys.sun_sky_sources(ghi=observed, dates=observed.index, normalisation=1, **Montpellier) 
sources = skys.add_sources(sun, sky)
#
cscene, raw, agg = ltfs.illuminate(lscene, light=sources, scene_unit='cm', north=180)
scene, _ = cscene.plot(raw)
skys.show_sources(sources,scene=scene, distance=15, radius=0.3, north=180)